# More Pandas

### Introduction
You have decided that you want to start your own animal shelter, but you want to get an idea of what that will entail and to get more information about planning. In this lecture, we'll look at a real data set collected by Austin Animal Center over several years and use our pandas skills from the last lecture and learn some new ones in order to explore this data further.

#### Our goals today are to be able to: <br/>

- Apply and use `.map()` and `.applymap()` from the Pandas library
- Explain what a groupby object is and split a DataFrame using `.groupby()`
- Explain lambda functions and use them on a DataFrame
- Reshape a DataFrame using joins, merges, pivoting, stacking, and melting
- Use one-hot encoding to make use of categorical variables

#### Getting started

Let's take a moment to download and to examine the [Austin Animal Center data set](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Outcomes/9t4d-g238/data). What kinds of questions can we ask this data and what kinds of information can we get back?

Let's take a look at the data:

In [1]:
import numpy as np
import pandas as pd
animals = pd.read_csv('Austin_Animal_Center_Outcomes.csv')
animals.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A720371,Moose,02/13/2016 05:59:00 PM,02/13/2016 05:59:00 PM,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
1,A674754,NaN,03/18/2014 11:47:00 AM,03/18/2014 11:47:00 AM,03/12/2014,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby
2,A689724,*Donatello,10/18/2014 06:52:00 PM,10/18/2014 06:52:00 PM,08/01/2014,Adoption,NaN,Cat,Neutered Male,2 months,Domestic Shorthair Mix,Black
3,A680969,*Zeus,08/05/2014 04:59:00 PM,08/05/2014 04:59:00 PM,06/03/2014,Adoption,NaN,Cat,Neutered Male,2 months,Domestic Shorthair Mix,White/Orange Tabby
4,A684617,NaN,07/27/2014 09:00:00 AM,07/27/2014 09:00:00 AM,07/26/2012,Transfer,SCRP,Cat,Intact Female,2 years,Domestic Shorthair Mix,Black


In [3]:
animals.shape

(114918, 12)

In [4]:
len(animals['Animal ID'].unique())

102837

What do we notice about this dataset?

In [5]:
animals.isnull()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,False,False,False,False,False,False,True,False,False,False,False,False
1,False,True,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,True,False,False,False,False,False
3,False,False,False,False,False,False,True,False,False,False,False,False
4,False,True,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
114913,False,True,False,False,False,False,False,False,False,False,False,False
114914,False,False,False,False,False,False,True,False,False,False,False,False
114915,False,True,False,False,False,False,True,False,False,False,False,False
114916,False,False,False,False,False,False,True,False,False,False,False,False


In [8]:
animals.isnull().sum()

Animal ID               0
Name                36075
DateTime                0
MonthYear               0
Date of Birth           0
Outcome Type            6
Outcome Subtype     62943
Animal Type             0
Sex upon Outcome        4
Age upon Outcome       27
Breed                   0
Color                   0
dtype: int64

In [9]:
animals.fillna('null')

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A720371,Moose,02/13/2016 05:59:00 PM,02/13/2016 05:59:00 PM,10/08/2015,Adoption,null,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
1,A674754,null,03/18/2014 11:47:00 AM,03/18/2014 11:47:00 AM,03/12/2014,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby
2,A689724,*Donatello,10/18/2014 06:52:00 PM,10/18/2014 06:52:00 PM,08/01/2014,Adoption,null,Cat,Neutered Male,2 months,Domestic Shorthair Mix,Black
3,A680969,*Zeus,08/05/2014 04:59:00 PM,08/05/2014 04:59:00 PM,06/03/2014,Adoption,null,Cat,Neutered Male,2 months,Domestic Shorthair Mix,White/Orange Tabby
4,A684617,null,07/27/2014 09:00:00 AM,07/27/2014 09:00:00 AM,07/26/2012,Transfer,SCRP,Cat,Intact Female,2 years,Domestic Shorthair Mix,Black
...,...,...,...,...,...,...,...,...,...,...,...,...
114913,A760365,null,10/18/2017 01:27:00 PM,10/18/2017 01:27:00 PM,10/17/2016,Transfer,Partner,Cat,Intact Female,1 year,Domestic Shorthair Mix,Silver Tabby
114914,A767465,Loco,03/01/2018 06:28:00 PM,03/01/2018 06:28:00 PM,03/01/2014,Return to Owner,null,Dog,Neutered Male,4 years,Chihuahua Shorthair Mix,Black/Cream
114915,A774386,null,06/23/2018 11:59:00 AM,06/23/2018 11:59:00 AM,04/07/2018,Adoption,null,Cat,Neutered Male,2 months,Domestic Shorthair Mix,Brown Tabby
114916,A772554,Muneca,05/21/2018 12:59:00 PM,05/21/2018 12:59:00 PM,11/01/2012,Return to Owner,null,Dog,Spayed Female,5 years,Norfolk Terrier Mix,Tan


In [10]:
animals.fillna(np.nan)

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A720371,Moose,02/13/2016 05:59:00 PM,02/13/2016 05:59:00 PM,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
1,A674754,NaN,03/18/2014 11:47:00 AM,03/18/2014 11:47:00 AM,03/12/2014,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby
2,A689724,*Donatello,10/18/2014 06:52:00 PM,10/18/2014 06:52:00 PM,08/01/2014,Adoption,NaN,Cat,Neutered Male,2 months,Domestic Shorthair Mix,Black
3,A680969,*Zeus,08/05/2014 04:59:00 PM,08/05/2014 04:59:00 PM,06/03/2014,Adoption,NaN,Cat,Neutered Male,2 months,Domestic Shorthair Mix,White/Orange Tabby
4,A684617,NaN,07/27/2014 09:00:00 AM,07/27/2014 09:00:00 AM,07/26/2012,Transfer,SCRP,Cat,Intact Female,2 years,Domestic Shorthair Mix,Black
...,...,...,...,...,...,...,...,...,...,...,...,...
114913,A760365,NaN,10/18/2017 01:27:00 PM,10/18/2017 01:27:00 PM,10/17/2016,Transfer,Partner,Cat,Intact Female,1 year,Domestic Shorthair Mix,Silver Tabby
114914,A767465,Loco,03/01/2018 06:28:00 PM,03/01/2018 06:28:00 PM,03/01/2014,Return to Owner,NaN,Dog,Neutered Male,4 years,Chihuahua Shorthair Mix,Black/Cream
114915,A774386,NaN,06/23/2018 11:59:00 AM,06/23/2018 11:59:00 AM,04/07/2018,Adoption,NaN,Cat,Neutered Male,2 months,Domestic Shorthair Mix,Brown Tabby
114916,A772554,Muneca,05/21/2018 12:59:00 PM,05/21/2018 12:59:00 PM,11/01/2012,Return to Owner,NaN,Dog,Spayed Female,5 years,Norfolk Terrier Mix,Tan


### 1. Applying and using map and applymap from the Pandas library

The Pandas library has several useful tools built in. Let's explore some of them.

#### DataFrame.applymap() and Series.map()

The ```.applymap()``` method takes a function as input that it will then apply to every entry in the dataframe.

In [11]:
animals.applymap(str).head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A720371,Moose,02/13/2016 05:59:00 PM,02/13/2016 05:59:00 PM,10/08/2015,Adoption,nan,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
1,A674754,nan,03/18/2014 11:47:00 AM,03/18/2014 11:47:00 AM,03/12/2014,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby
2,A689724,*Donatello,10/18/2014 06:52:00 PM,10/18/2014 06:52:00 PM,08/01/2014,Adoption,nan,Cat,Neutered Male,2 months,Domestic Shorthair Mix,Black
3,A680969,*Zeus,08/05/2014 04:59:00 PM,08/05/2014 04:59:00 PM,06/03/2014,Adoption,nan,Cat,Neutered Male,2 months,Domestic Shorthair Mix,White/Orange Tabby
4,A684617,nan,07/27/2014 09:00:00 AM,07/27/2014 09:00:00 AM,07/26/2012,Transfer,SCRP,Cat,Intact Female,2 years,Domestic Shorthair Mix,Black


The .map() method takes a function as input that it will then apply to every entry in the Series.

In [12]:
# This line of code will split the IDs into two parts and add the parts as new columns.

animals[['Animal ID Prefix', 'Animal ID Num']] =\
animals['Animal ID'].str.split('A', expand=True)
animals.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color,Animal ID Prefix,Animal ID Num
0,A720371,Moose,02/13/2016 05:59:00 PM,02/13/2016 05:59:00 PM,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff,,720371
1,A674754,NaN,03/18/2014 11:47:00 AM,03/18/2014 11:47:00 AM,03/12/2014,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby,,674754
2,A689724,*Donatello,10/18/2014 06:52:00 PM,10/18/2014 06:52:00 PM,08/01/2014,Adoption,NaN,Cat,Neutered Male,2 months,Domestic Shorthair Mix,Black,,689724
3,A680969,*Zeus,08/05/2014 04:59:00 PM,08/05/2014 04:59:00 PM,06/03/2014,Adoption,NaN,Cat,Neutered Male,2 months,Domestic Shorthair Mix,White/Orange Tabby,,680969
4,A684617,NaN,07/27/2014 09:00:00 AM,07/27/2014 09:00:00 AM,07/26/2012,Transfer,SCRP,Cat,Intact Female,2 years,Domestic Shorthair Mix,Black,,684617


In [13]:
# Now: How can we convert the Animal ID Num column to integers?

animals['Animal ID Num'] = animals['Animal ID Num'].map(int)

Or we could have just used the `.astype()` method:

In [14]:
animals['Animal ID Num'] = animals['Animal ID Num'].astype(int)

#### Anonymous Functions (Lambda Abstraction)

Simple functions can be defined right in the function call. This is called 'lambda abstraction'; the function thus defined has no name and hence is "anonymous".

In [15]:
animals['Animal ID Num'].map(lambda x: x*2)[:4]

0    1440742
1    1349508
2    1379448
3    1361938
Name: Animal ID Num, dtype: int64

**Exercise: Use an anonymous function to add 'approximately' in front of the entries in Age upon Outcome**

In [21]:
# Your code here!
animals['Age upon Outcome'].map(lambda x: 'approximately ' + str(x))[:4]


0    approximately 4 months
1      approximately 6 days
2    approximately 2 months
3    approximately 2 months
Name: Age upon Outcome, dtype: object

What went wrong? How can we fix it?

### 2. Methods for Re-Organizing DataFrames: .groupby()

Those of you familiar with SQL have probably used the GROUP BY command. (And if you haven't, you'll see it very soon!) Pandas has this, too.

The .groupby() method is especially useful for aggregate functions applied to the data grouped in particular ways.

In [23]:
animals.groupby('Animal Type')

#### .groups and .get_group()

In [24]:
animals.groupby('Animal Type').groups

{'Bird': Int64Index([    44,    145,    348,    634,   1060,   1473,   1696,   1716,
               1811,   2068,
             ...
             113313, 113433, 113566, 113896, 114075, 114328, 114611, 114661,
             114759, 114761],
            dtype='int64', length=539),
 'Cat': Int64Index([     1,      2,      3,      4,      5,      6,      7,      9,
                 11,     14,
             ...
             114888, 114891, 114893, 114899, 114904, 114905, 114906, 114913,
             114915, 114917],
            dtype='int64', length=43294),
 'Dog': Int64Index([     0,     10,     12,     13,     15,     16,     18,     19,
                 20,     22,
             ...
             114900, 114901, 114902, 114903, 114908, 114910, 114911, 114912,
             114914, 114916],
            dtype='int64', length=65186),
 'Livestock': Int64Index([   624,   1569,  20834,  24987,  29678,  50081,  54044,  54397,
              62044,  66195,  77429,  81079,  81429,  84775,  91787,  9205

In [25]:
animals.groupby('Animal Type').get_group('Livestock')

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color,Animal ID Prefix,Animal ID Num
624,A795191,Loki,05/18/2019 03:37:00 PM,05/18/2019 03:37:00 PM,11/17/2018,Return to Owner,NaN,Livestock,Intact Male,5 months,Pig,White,,795191
1569,A668167,NaN,11/30/2013 12:18:00 PM,11/30/2013 12:18:00 PM,05/28/2013,Return to Owner,NaN,Livestock,Intact Female,6 months,Pig Mix,Black/White,,668167
20834,A673651,NaN,03/11/2014 02:39:00 PM,03/11/2014 02:39:00 PM,02/28/2013,Adoption,Foster,Livestock,Neutered Male,1 year,Pig Mix,Black/White,,673651
24987,A718910,NaN,01/27/2016 12:00:00 AM,01/27/2016 12:00:00 AM,01/09/2015,Transfer,Partner,Livestock,Intact Male,1 year,Pig Mix,White,,718910
29678,A803469,NaN,09/08/2019 08:00:00 AM,09/08/2019 08:00:00 AM,09/01/2017,Return to Owner,NaN,Livestock,Intact Female,2 years,Pygmy,Tan/Black,,803469
50081,A701250,NaN,05/11/2015 12:00:00 AM,05/11/2015 12:00:00 AM,04/26/2013,Transfer,Partner,Livestock,Intact Female,2 years,Pig Mix,Pink,,701250
54044,A674214,NaN,03/29/2014 02:00:00 PM,03/29/2014 02:00:00 PM,02/22/2014,Adoption,Foster,Livestock,Unknown,5 weeks,Pig Mix,Black,,674214
54397,A811675,NaN,01/08/2020 09:41:00 AM,01/08/2020 09:41:00 AM,01/07/2018,NaN,NaN,Livestock,Intact Female,NaN,Goat,Black/White,,811675
62044,A715047,NaN,12/07/2015 12:00:00 AM,12/07/2015 12:00:00 AM,10/30/2014,Transfer,Partner,Livestock,Unknown,1 year,Goat Mix,Brown,,715047
66195,A663228,NaN,10/03/2013 10:59:00 AM,10/03/2013 10:59:00 AM,09/15/2008,Transfer,Partner,Livestock,Intact Male,5 years,Miniature,Liver/Cream,,663228


#### Aggregating

In [31]:
animals.groupby('Animal Type').max()

,Animal ID,DateTime,MonthYear,Date of Birth,Breed,Color,Animal ID Prefix,Animal ID Num
Animal Type,,,,,,,,
Bird,A811990,12/31/2016 01:04:00 PM,12/31/2016 01:04:00 PM,12/31/2016,Wren Mix,Yellow/White,,811990
Cat,A812956,12/31/2019 12:19:00 PM,12/31/2019 12:19:00 PM,12/31/2018,Turkish Van Mix,Yellow/Orange Tabby,,812956
Dog,A813026,12/31/2019 12:57:00 PM,12/31/2019 12:57:00 PM,12/31/2018,Yorkshire Terrier/Yorkshire Terrier,Yellow/Yellow,,813026
Livestock,A811675,12/07/2015 12:00:00 AM,12/07/2015 12:00:00 AM,11/22/2013,Sheep Mix,White/Black,,811675
Other,A813028,12/31/2019 03:00:00 PM,12/31/2019 03:00:00 PM,12/31/2018,Turtle Mix,Yellow/Black,,813028



#### Datetime Objects

'Datetime' is a special data type for dates. And we can convert an appropriately formatted variable to the datetime type simply by calling `pd.to_datetime()`.

In [32]:
pd.to_datetime(animals['Date of Birth'])

0        2015-10-08
1        2014-03-12
2        2014-08-01
3        2014-06-03
4        2012-07-26
            ...    
114913   2016-10-17
114914   2014-03-01
114915   2018-04-07
114916   2012-11-01
114917   2017-07-08
Name: Date of Birth, Length: 114918, dtype: datetime64[ns]

**Exercise: Find the latest date of birth per animal type.**

In [41]:
# First redefine Date of Birth as a series of datetime objects.
# Then group by Animal Type and calculate the max.
animals['datetime'] = pd.to_datetime(animals['Date of Birth'])

animals.groupby('Animal Type').max()

,Animal ID,DateTime,MonthYear,Date of Birth,Breed,Color,Animal ID Prefix,Animal ID Num,datetime
Animal Type,,,,,,,,,
Bird,A811990,12/31/2016 01:04:00 PM,12/31/2016 01:04:00 PM,12/31/2016,Wren Mix,Yellow/White,,811990,2019-11-02
Cat,A812956,12/31/2019 12:19:00 PM,12/31/2019 12:19:00 PM,12/31/2018,Turkish Van Mix,Yellow/Orange Tabby,,812956,2020-01-09
Dog,A813026,12/31/2019 12:57:00 PM,12/31/2019 12:57:00 PM,12/31/2018,Yorkshire Terrier/Yorkshire Terrier,Yellow/Yellow,,813026,2020-01-04
Livestock,A811675,12/07/2015 12:00:00 AM,12/07/2015 12:00:00 AM,11/22/2013,Sheep Mix,White/Black,,811675,2018-11-17
Other,A813028,12/31/2019 03:00:00 PM,12/31/2019 03:00:00 PM,12/31/2018,Turtle Mix,Yellow/Black,,813028,2019-12-21


### 3. Reshaping a DataFrame

#### .pivot()

Those of you familiar with Excel have probably used Pivot Tables. Pandas has a similar functionality.

In [42]:
animals.pivot(values='Age upon Outcome', columns='Animal Type').head()

Animal Type,Bird,Cat,Dog,Livestock,Other
0,NaN,NaN,4 months,NaN,NaN
1,NaN,6 days,NaN,NaN,NaN
2,NaN,2 months,NaN,NaN,NaN
3,NaN,2 months,NaN,NaN,NaN
4,NaN,2 years,NaN,NaN,NaN


### 4. Methods for Combining DataFrames: .join(), .merge(), .concat(), .melt()

#### .join()

In [43]:
toy1 = pd.DataFrame([[63, 142], [33, 47]], columns=['age', 'HP'])
toy2 = pd.DataFrame([[63, 100], [33, 200]], columns=['age', 'MP'])

In [44]:
toy1

,age,HP
0,63,142
1,33,47


In [45]:
toy2

,age,MP
0,63,100
1,33,200


In [46]:
toy1.set_index('age').join(toy2.set_index('age'))

,HP,MP
age,,
63,142,100
33,47,200


For more on this method, check out the [doc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html)!

#### .merge()

In [47]:
ds_chars = pd.read_csv('ds_chars.csv', index_col=0)
ds_chars

,name,HP,home_state
0,greg,200,WA
1,miles,200,WA
2,alan,170,TX
3,alison,300,DC
4,rachel,200,TX


In [48]:
states = pd.read_csv('states.csv', index_col=0)
states

,state,nickname,capital
0,WA,evergreen,Olympia
1,TX,alamo,Austin
2,DC,district,Washington
3,OH,buckeye,Columbus
4,OR,beaver,Salem


In [51]:
ds_chars.merge(states, left_on='home_state', right_on='state', how='left')

,name,HP,home_state,state,nickname,capital
0,greg,200,WA,WA,evergreen,Olympia
1,miles,200,WA,WA,evergreen,Olympia
2,alan,170,TX,TX,alamo,Austin
3,alison,300,DC,DC,district,Washington
4,rachel,200,TX,TX,alamo,Austin


#### pd.concat()

This method takes a *list* of pandas objects as arguments.

N.B. The cell below will likely produce a **Deprecation Warning**.

In [52]:
ds_full = pd.concat([ds_chars, states])
ds_full

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,HP,capital,home_state,name,nickname,state
0,200.0,NaN,WA,greg,NaN,NaN
1,200.0,NaN,WA,miles,NaN,NaN
2,170.0,NaN,TX,alan,NaN,NaN
3,300.0,NaN,DC,alison,NaN,NaN
4,200.0,NaN,TX,rachel,NaN,NaN
0,NaN,Olympia,NaN,NaN,evergreen,WA
1,NaN,Austin,NaN,NaN,alamo,TX
2,NaN,Washington,NaN,NaN,district,DC
3,NaN,Columbus,NaN,NaN,buckeye,OH
4,NaN,Salem,NaN,NaN,beaver,OR


`pd.concat()`––and many other pandas operations––make use of an `axis` parameter. For this particular method I need to specify whether I want to concatenate the DataFrames *row-wise* (`axis=0`) or *column-wise* (`axis=1`). The default is `axis=0`, so let's override that!

In [54]:
pd.concat([ds_chars, states], axis=1)

,name,HP,home_state,state,nickname,capital
0,greg,200,WA,WA,evergreen,Olympia
1,miles,200,WA,TX,alamo,Austin
2,alan,170,TX,DC,district,Washington
3,alison,300,DC,OH,buckeye,Columbus
4,rachel,200,TX,OR,beaver,Salem


#### pd.melt()

Melting removes the structure from your DataFrame and puts the data in a 'variable' and 'value' format.

In [55]:
pd.melt(ds_full)

,variable,value
0,HP,200
1,HP,200
2,HP,170
3,HP,300
4,HP,200
5,HP,NaN
6,HP,NaN
7,HP,NaN
8,HP,NaN
9,HP,NaN


### 5. Making Use of Categories: One-Hot Encoding

Pandas has a one-hot encoder called `get_dummies()`, which is good for exploratory data analysis (EDA).

This might be good to use if we're in the **data-understanding** stage (Stage 2) of our CRISP-DM process.

We can call it on a DataFrame as a whole or on a Series (column).

In [56]:
pd.get_dummies(animals['Animal Type'])

,Bird,Cat,Dog,Livestock,Other
0,0,0,1,0,0
1,0,1,0,0,0
2,0,1,0,0,0
3,0,1,0,0,0
4,0,1,0,0,0
...,...,...,...,...,...
114913,0,1,0,0,0
114914,0,0,1,0,0
114915,0,1,0,0,0
114916,0,0,1,0,0


If however we're in a later stage of the process and we're interested, say, in preparing a data pipeline, `pandas.get_dummies()` will prove inferior to other tools.

In practice, we will **not** use `pandas.get_dummies()`. The library Scikit-Learn (`sklearn`, included with your Anaconda installation) has a `OneHotEncoder` class that creates an object that persists. This makes it much more apt for production environments, and so it's good to get in the habit of using it.

Ultimately, we will use **many** tools from sklearn.

In [57]:
from sklearn.preprocessing import OneHotEncoder

In [58]:
ohe = OneHotEncoder()

In [59]:
ohe.fit(animals[['Animal Type']])

OneHotEncoder(categorical_features=None, categories=None, drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='error',
              n_values=None, sparse=True)

Now that the `OneHotEncoder` has been fitted to our data, it has newly available attributes and methods. In particular, it has access to the different categories that we're replacing:

In [60]:
ohe.get_feature_names()

array(['x0_Bird', 'x0_Cat', 'x0_Dog', 'x0_Livestock', 'x0_Other'],
      dtype=object)

We'll have much more to say about `sklearn` syntax and about Python's object structure. But let's now transform our data to see what the new table looks like:

In [61]:
ohe.transform(animals[['Animal Type']])

<114918x5 sparse matrix of type '<class 'numpy.float64'>'
	with 114918 stored elements in Compressed Sparse Row format>

For the sake of saving storage space, the return is a **sparse matrix**, but we can "re-inflate it if we want to see it in tabular form:

In [62]:
types_encoded = ohe.transform(animals[['Animal Type']]).todense()
types_encoded

matrix([[0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        ...,
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

Let's put it into a DataFrame:

In [70]:
what = pd.DataFrame(types_encoded, columns=ohe.get_feature_names())

In [75]:
animals
pd.concat([animals, pd.DataFrame(types_encoded, columns=ohe.get_feature_names())], axis=1)

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color,Animal ID Prefix,Animal ID Num,datetime,x0_Bird,x0_Cat,x0_Dog,x0_Livestock,x0_Other
0,A720371,Moose,02/13/2016 05:59:00 PM,02/13/2016 05:59:00 PM,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff,,720371,2015-10-08,0.0,0.0,1.0,0.0,0.0
1,A674754,NaN,03/18/2014 11:47:00 AM,03/18/2014 11:47:00 AM,03/12/2014,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby,,674754,2014-03-12,0.0,1.0,0.0,0.0,0.0
2,A689724,*Donatello,10/18/2014 06:52:00 PM,10/18/2014 06:52:00 PM,08/01/2014,Adoption,NaN,Cat,Neutered Male,2 months,Domestic Shorthair Mix,Black,,689724,2014-08-01,0.0,1.0,0.0,0.0,0.0
3,A680969,*Zeus,08/05/2014 04:59:00 PM,08/05/2014 04:59:00 PM,06/03/2014,Adoption,NaN,Cat,Neutered Male,2 months,Domestic Shorthair Mix,White/Orange Tabby,,680969,2014-06-03,0.0,1.0,0.0,0.0,0.0
4,A684617,NaN,07/27/2014 09:00:00 AM,07/27/2014 09:00:00 AM,07/26/2012,Transfer,SCRP,Cat,Intact Female,2 years,Domestic Shorthair Mix,Black,,684617,2012-07-26,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114913,A760365,NaN,10/18/2017 01:27:00 PM,10/18/2017 01:27:00 PM,10/17/2016,Transfer,Partner,Cat,Intact Female,1 year,Domestic Shorthair Mix,Silver Tabby,,760365,2016-10-17,0.0,1.0,0.0,0.0,0.0
114914,A767465,Loco,03/01/2018 06:28:00 PM,03/01/2018 06:28:00 PM,03/01/2014,Return to Owner,NaN,Dog,Neutered Male,4 years,Chihuahua Shorthair Mix,Black/Cream,,767465,2014-03-01,0.0,0.0,1.0,0.0,0.0
114915,A774386,NaN,06/23/2018 11:59:00 AM,06/23/2018 11:59:00 AM,04/07/2018,Adoption,NaN,Cat,Neutered Male,2 months,Domestic Shorthair Mix,Brown Tabby,,774386,2018-04-07,0.0,1.0,0.0,0.0,0.0
114916,A772554,Muneca,05/21/2018 12:59:00 PM,05/21/2018 12:59:00 PM,11/01/2012,Return to Owner,NaN,Dog,Spayed Female,5 years,Norfolk Terrier Mix,Tan,,772554,2012-11-01,0.0,0.0,1.0,0.0,0.0


In [ ]:
pd.concat([animals], axis=1)